In [1]:
from openpyxl import load_workbook
import os
import numpy as np
from numpy.random import randn
import pandas as pd
from pandas import Series, DataFrame
from dateutil.parser import parse
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import csv
import datetime
import time

In [2]:
df_list = [
    #df1 = pd.read_csv('202107_TFM.csv')
    #df2 = pd.read_csv('202108_TFM.csv')
    #df3 = pd.read_csv('202109_TFM.csv')
    #pd.read_csv('202110_TFM.csv'),
    #pd.read_csv('202111_TFM.csv'),
    #pd.read_csv('202112_TFM.csv'),
    #pd.read_csv('202201_TFM.csv'),
    #pd.read_csv('202202_TFM.csv'),
    #pd.read_csv('202203_TFM.csv'),
    #pd.read_csv('202204_TFM.csv'),
    #pd.read_csv('202205_TFM.csv'),
    #pd.read_csv('202206_TFM.csv')
    #pd.read_csv('202207_TFM.csv'),
    #pd.read_csv('202208_TFM.csv'),
    pd.read_csv('202209_TFM.csv'),
    pd.read_csv('202210_TFM.csv'),
    pd.read_csv('202211_TFM.csv'),
    pd.read_csv('202212_TFM.csv'),
    pd.read_csv('202301_TFM.csv'),
    pd.read_csv('202302_TFM.csv')
    #pd.read_csv('202303_TFM.csv')
]

In [3]:
df = pd.concat(df_list)

In [4]:
m_list = [
    pd.read_csv('202205~202301TFM.csv'),
    pd.read_csv('202302_TFM2.csv'),
    pd.read_csv('202303_TFM2.csv')
]

In [5]:
pq = pd.read_csv('EQ_Process_BN.csv')

In [6]:
pq['EQ_ID'] = pq['EQ_ID'].str.strip()

In [7]:
m = pd.concat(m_list)

In [8]:
m.rename(columns={'Ident':'Metakey Ident'},inplace=True)

In [9]:
device = pd.read_csv('Device.csv')

In [10]:
df = pd.merge(df, m[['Metakey Ident','Device']], on="Metakey Ident", how='left')

In [11]:
df = pd.merge(df, device, on="Device", how='left')

In [12]:
df = pd.merge(df, m[['Metakey Ident','Lotid']], on="Metakey Ident", how='left')

In [13]:
EQ_ident = pd.read_csv('EQ_ident.csv')

In [14]:
EQ_states = pd.read_csv('EQ_states.csv')

In [15]:
Lv5_Indent = pd.read_csv('Lv5_indent.csv')

In [16]:
Lv5_Indent.rename(columns={'STATE_IDENT':'Tr25 5 Status'},inplace=True)

In [17]:
df = pd.merge(df, EQ_ident, on='Eq Ident', how = 'inner')

In [18]:
df['Process'] = df['EQ_ID'].str.extract(pat = '([A-Z]..)')

In [19]:
df = pd.merge(df, Lv5_Indent, on='Tr25 5 Status', how = 'inner')

In [20]:
df = pd.merge(df, EQ_states, on='Eq Status', how = 'outer')

In [21]:
df = df.dropna(subset=['Timestamp End', 'Timestamp Start'])

In [22]:
df['Timestamp Start'] = pd.to_datetime(df['Timestamp Start'], format='%Y-%m-%d %p %I:%M:%S', errors='ignore')

In [23]:
def convert_to_24h_format(dt_str):
    parts = dt_str.split(' ')
    date = parts[0]
    time_parts = parts[2].split(':')
    hours = int(time_parts[0])
    if parts[1] == '오후' and hours != 12:
        hours += 12
    elif parts[1] == '오전' and hours == 12:
        hours = 0
    time = f'{hours:02d}:{time_parts[1]}:{time_parts[2]}'
    return f'{date} {time}'

In [24]:
convert_to_24h_format_v = np.vectorize(convert_to_24h_format)
df['Timestamp Start'] = convert_to_24h_format_v(df['Timestamp Start'].values)
df['Timestamp Start'] = pd.to_datetime(df['Timestamp Start'], format='%Y-%m-%d %H:%M:%S')
#df['Timestamp Start'] = df['Timestamp Start'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [25]:
convert_to_24h_format_v = np.vectorize(convert_to_24h_format)
df['Timestamp End'] = convert_to_24h_format_v(df['Timestamp End'].values)
df['Timestamp End'] = pd.to_datetime(df['Timestamp End'], format='%Y-%m-%d %H:%M:%S')
#df['Timestamp End'] = df['Timestamp End'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [26]:
df['seconds'] = (df['Timestamp End'] - df['Timestamp Start']).dt.total_seconds().astype(int)

In [27]:
df['Month'] = df['Timestamp Start'].dt.strftime('%Y-%m')

In [28]:
#df['Time'] = df['Timestamp Start'].dt.strftime("%H:%M:%S")
#df['Date'] = pd.to_datetime(df['Timestamp Start']).dt.to_period('D')

In [28]:
df = df[['Month','Timestamp Start','Process','EQ_ID', 'STATE_NAME_x', 'STATE_NAME_y','seconds','Product type', 'Lotid']]

In [29]:
df.sort_values(by=['EQ_ID', 'Timestamp Start'], inplace=True)

In [30]:
df['Lotid'] = df['Lotid'].fillna(method = 'ffill')

In [31]:
df['Product type'] = df['Product type'].fillna(method = 'ffill')

In [32]:
df = df[['Month', 'Process','EQ_ID', 'Lotid', 'Product type','STATE_NAME_x', 'STATE_NAME_y','seconds',]]

In [33]:
df.head()

,Month,Process,EQ_ID,Lotid,Product type,STATE_NAME_x,STATE_NAME_y,seconds
3621161,2022-09,AFV,AFVI001,NaN,NaN,Normal Production,Normal Production,901
1154688,2022-09,AFV,AFVI001,NaN,NaN,Undefined Waiting,Undefined Waiting,49
3621164,2022-09,AFV,AFVI001,NaN,NaN,Normal Production,Normal Production,23
3621165,2022-09,AFV,AFVI001,2Z2340S8,IKCM10H60GA,Normal Production,Normal Production,23
3621166,2022-09,AFV,AFVI001,2Z2340S8,IKCM10H60GA,Normal Production,Normal Production,809


In [34]:
df['group'] = (df['STATE_NAME_y'] != df['STATE_NAME_y'].shift()).cumsum()
df2 = df.groupby('group').agg({'Month': 'first', 'Process': 'first', 'EQ_ID': 'first', 'Lotid': 'first' , 'Product type': 'first', 'STATE_NAME_x': 'first', 'STATE_NAME_y': 'first', 'seconds': 'sum'})
df2 = df2.reset_index(drop=False)

In [35]:
df2 = df2.drop(['group'], axis=1)

In [39]:
one_page_MMO(df2, pq)

In [38]:
def one_page_MMO(df2, pq):
    

    def append_df_to_excel(filename, k, sheet_name='Sheet1', startrow=None,
                           truncate_sheet=False,
                           **to_excel_kwargs):


        # ignore [engine] parameter if it was passed
        if 'engine' in to_excel_kwargs:
            to_excel_kwargs.pop('engine')

        writer = pd.ExcelWriter(filename, engine='openpyxl', mode='a',if_sheet_exists='overlay')
        
        try:
            FileNotFoundError
        except NameError:
            FileNotFoundError = IOError
            
        try:
            # try to open an existing workbook
            writer.book = load_workbook(filename)

            # get the last row in the existing Excel sheet
            # if it was not specified explicitly
            if startrow is None and sheet_name in writer.book.sheetnames:
                startrow = writer.book[sheet_name].max_row

            # truncate sheet
            if truncate_sheet and sheet_name in writer.book.sheetnames:
                # index of [sheet_name] sheet
                idx = writer.book.sheetnames.index(sheet_name)
                # remove [sheet_name]
                writer.book.remove(writer.book.worksheets[idx])
                # create an empty sheet [sheet_name] using old index
                writer.book.create_sheet(sheet_name, idx)

            # copy existing sheets
            writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
        except FileNotFoundError:
            # file does not exist yet, we will create it
            pass
            
        if startrow is None:
            startrow = 0
        #with pd.ExcelWriter('recent_OEE3.xlsx', mode='a') as writer:
        k.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)
        
        writer.save()
        
    filename = r'C:\Users\ParkPaul\Documents\Python\UDW_UW_T.xlsx'

    for index, row in pq.iterrows():
        k = row
        lst = row['EQ_ID'].split(',')
        lst = [x.strip(' ') for x in lst]
        df3 = df2[df2['EQ_ID'].isin(lst)]
        
        a=[]
        try:
            UW = len(df3[df3['STATE_NAME_x'] == 'Undefined Waiting'])
            T = len(df3[df3['STATE_NAME_x'] == 'Waiting For Technician'])
            a.append((UW)/(df3.groupby(['Lotid']).count().shape[0]))
            a.append((T)/(df3.groupby(['Lotid']).count().shape[0]))
            a.append((df3[df3['STATE_NAME_x'] == 'Undefined Waiting']['seconds'].sum())/(UW))
            a.append((df3[df3['STATE_NAME_x'] == 'Waiting For Technician']['seconds'].sum())/(T))
            
        except ZeroDivisionError:
            pass
        

        a = pd.DataFrame(a)
        a = a.transpose()
        
        a = a.rename(columns={0: row['Device'] + "_" + row['Process']})


        append_df_to_excel(filename, a)

In [36]:
df['Product type'].isnull().values.any()

False

In [36]:
PFC_D = ['IM564-X6D','IFCM30T65GD','IFCM20T65GD', 'IFCM15S60GS', 'IFCM15P60GD', 'IFCM15S60GD']
PFC_EQ = ['LSD011', 'LSD017','LSD028', 'LSD029', 'LSD040', 'LSD039']
PFC_EQ2 = ['LSD011', 'LSD027','LSD029', 'LSD038']
#df.sort_values(by=['EQ_ID','Timestamp Start'], inplace=True)
#df['Product type'] = np.where(df['Product type'].fillna(method = 'bfill'), df['Product type'])
#df['Product type'] = np.where(df['Product type'].fillna(method = 'ffill'), df['Product type'])

df2 = df2[~( (df2['Product type'].isin(PFC_D)) & (df2['EQ_ID'].isin(PFC_EQ))) ]
df2 = df2[~( (~df2['Product type'].isin(PFC_D)) & (df2['EQ_ID'].isin(PFC_EQ2))) ]

In [ ]:
#Nomaterial 앞에 있는 Waiting for response 빼고 계산법
df2 = df2.reset_index()
k = df2.index[df2['STATE_NAME_x'] == "No Material" ].tolist()
lst=[]
w=[]
for i in k:
    if df2.iloc[i-1]['STATE_NAME_x'] == "Waiting for Response":
        lst.append(i-1)

w = lst + k
df2 = df2[~df2.index.isin(w)]
df2 = df2[(df2.STATE_NAME_x != 'No Material')]

In [25]:
filter1 = df2['EQ_ID'].isin(['LSD001', 'LSD002', 'LSD003', 'LSD004', 'LSD005', 'LSD006', 'LSD007', 'LSD008', 'LSD009', 'LSD010', 'LSD011', 'LSD012', 'LSD013', 'LSD014', 'LSD015', 'LSD016', 'LSD017', 'LSD018', 'LSD019', 'LSD020', 'LSD021', 'LSD022', 'LSD023', 'LSD024', 'LSD025', 'LSD026', 'LSD027', 'LSD028', 'LSD029', 'LSD030', 'LSD031', 'LSD035', 'LSD036', 'LSD037', 'LSD038', 'LSD039', 'LSD040', 'LSD041', 'LSD042'])

In [26]:
filter2 = df2['EQ_ID'].isin(['LSD001', 'LSD003', 'LSD004', 'LSD005', 'LSD006', 'LSD007', 'LSD009', 'LSD010', 'LSD012', 'LSD013', 'LSD014', 'LSD015', 'LSD016', 'LSD018', 'LSD019', 'LSD020', 'LSD021', 'LSD023', 'LSD024', 'LSD025', 'LSD026', 'LSD027', 'LSD030', 'LSD031', 'LSD035', 'LSD036', 'LSD037', 'LSD038', 'LSD041'])

In [25]:
#Al WB all
filter1 = df2['EQ_ID'].isin(['LAW004', 'LAW006', 'LAW007', 'LAW008', 'LAW009', 'LAW010', 'LAW011', 'LAW012', 'LAW013', 'LAW016', 'LAW018', 'LAW020', 'LAW022', 'LAW024', 'LAW026', 'LAW030', 'LAW033', 'LAW034', 'LAW035', 'LAW036', 'LAW040', 'LAW043', 'LAW050', 'LAW028', 'LAW029', 'LAW032', 'LAW038', 'LAW045', 'LAW046', 'LAW053', 'LAW014', 'LAW037', 'LAW042', 'LAW044', 'LAW052', 'LAW041', 'LAW047', 'LAW031', 'LAW039', 'LAW049', 'LAW048', 'LAW051'])

In [42]:
#FP Al WB
filter1 = df2['EQ_ID'].isin(['LAW004','LAW005', 'LAW006', 'LAW007', 'LAW008', 'LAW009', 'LAW010', 'LAW011', 'LAW012', 'LAW013', 'LAW016', 'LAW018', 'LAW020', 'LAW022', 'LAW024', 'LAW026', 'LAW030', 'LAW033', 'LAW034', 'LAW035', 'LAW036', 'LAW040', 'LAW043'])

In [58]:
#DCG P&G
filter1 = df2['EQ_ID'].isin(['LAW029', 'LAW028', 'LAW038', 'LAW032', 'LAW046', 'LAW045', 'LAW053'])

In [78]:
#Tiny1.0 3x
filter1 = df2['EQ_ID'].isin(['LAW031', 'LAW039', 'LAW049'])

In [37]:
#BN SDA
filter1 = df2['EQ_ID'].isin(['LSD003', 'LSD004', 'LSD005', 'LSD006', 'LSD007', 'LSD009', 'LSD010', 'LSD013', 'LSD014', 'LSD015', 'LSD016', 'LSD018', 'LSD019', 'LSD024', 'LSD025', 'LSD026', 'LSD036', 'LSD037','LSD001', 'LSD011', 'LSD012', 'LSD017', 'LSD023', 'LSD027', 'LSD028', 'LSD029', 'LSD035', 'LSD038', 'LSD039', 'LSD040','LSD020', 'LSD021', 'LSD030', 'LSD031','LSD041'])

In [46]:
filter1 = df2['EQ_ID'].isin(['LMD002', 'LMD003', 'LMD004', 'LMD005', 'LMD006', 'LMD007', 'LMD009', 'LMD010', 'LMD011', 'LMD012', 'LMD013', 'LMD14'])

In [48]:
filter3 = df2['EQ_ID'].isin(['LSD020'])

In [59]:
df3 = df2[filter1]

In [38]:
df4 = pd.crosstab([df3.STATE_NAME_x],df3.Month,values=df3.seconds,aggfunc=sum,rownames=['Level 5'],colnames=['month'], normalize='columns')

In [39]:
df4.to_excel("Overall SDA.xlsx")

In [40]:
#df2[df2['STATE_NAME_x'].isin(['Normal Production'])].groupby(['Month']).sum()/3600/30/17

# Tool change & Spanker Cleaning & Mold cleaning

In [27]:
df4 = df3[df3['STATE_NAME_x'].isin(['Waiting for Response'])]

In [39]:
df4 = df3[df3['STATE_NAME_y'].isin(['Change Collet'])]

In [48]:
df4 = df3[df3['STATE_NAME_y'].isin(['Regular Mold Cleaning'])]

In [61]:
df4 = df3[df3['STATE_NAME_x'].isin(['Undefined Waiting'])]

In [80]:
df4 = df3[df3['STATE_NAME_y'].isin(['Change Tool'])]

In [66]:
df4 = df3[df3['STATE_NAME_y'].isin(['Waiting (Tool change OP)'])]

In [34]:
df3[df3['STATE_NAME_x'].isin(['Undefined Waiting'])]

,group,Month,Process,EQ_ID,STATE_NAME_x,STATE_NAME_y,seconds


In [40]:
df4 = df4[(df4.seconds > 180)] #Tool Change

In [41]:
df4 = df4[(df4.seconds < 5400)] # Tool Change

In [49]:
df4 = df4[(df4.seconds > 600)] # Mold Cleaning

In [50]:
df4 = df4[(df4.seconds < 7200)] #Mold Cleaning

In [69]:
df4 = df4[(df4.seconds < 3600)]

In [59]:
df4 = df4[(df4.Month == '2023-01')]

In [56]:
df4.sort_values(by=['seconds'], inplace=True)

In [58]:
#df3.groupby(['Month','EQ_ID'])[['Process']].count().reset_index().groupby(['Month'])[['EQ_ID']].count()

In [59]:
#df4.groupby(['EQ_ID']).sum().shape[0]

In [70]:
k = pd.crosstab([df4.STATE_NAME_y], df4.Month,values=df4.seconds,aggfunc='count',rownames=['LV6'],colnames=['month'])

In [99]:
k

month,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01
LV6,,,,,,,,,
Change Collet,1415,1478,1623,1230,1052,791,890,893,888


In [57]:
k/12

month,2023-01,2023-02,2023-03
LV6,,,
Regular Mold Cleaning,22.083333,21.166667,30.666667


In [71]:
v = pd.crosstab([df4.STATE_NAME_y], df4.Month,values=df4.seconds,aggfunc=sum,rownames=['LV6'],colnames=['month'])

In [86]:
v

month,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01
LV6,,,,,,,,,
Change Tool,150441.0,250291.0,395930.0,363395.0,227921.0,329855.0,444918.0,264445.0,290062.0


In [132]:
v/3600

month,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01
LV6,,,,,,,,,
Change Collet,71.328056,83.293056,106.864167,94.908611,66.095556,115.627222,133.7775,92.686667,81.301944


In [79]:
(v/k/60).mean(axis=1)

LV6
Undefined Waiting    0.671077
dtype: float64